In [ ]:
library(tidyverse)
library(tidymodels)
library(dplyr)

In [ ]:
sessions <- read_csv("data/sessions.csv")
players <- read_csv("data/players.csv")


In [ ]:
sessions|> summarise(average_original_start_time=mean(original_start_time,na.rm=TRUE), average_original_end_time=mean(original_end_time,na.rm=TRUE))|>round(2)
players |> summarise(average_played_hours=mean(played_hours, na.rm=TRUE), average_Age=mean(Age,na.rm=TRUE))|>round(2)


(1) Data Description

The Data Description of sessions.csv  
1.Number of Observation:1535  
2.Summary Statistics:  
-Average of original_satrt_time:1.719201e+12  
-Average of original_end_time:1.719196e+12  
3.NUmber of Variables:5  
4.Name, Type of Variables, and What the Variables Mean:  
-hashedEmail(character):Hashed version of player email used as an anonymous ID  
-start_time(character):Session start timestamp  
-end_time(character):Session end timestamp  
-original_start_time(double):Session start timestamp in Unix epoch milliseconds (UTC)  
-original_end_time(double):Session end timestamp in Unix epoch milliseconds (UTC)  

The Data Descripton of players.csv  
1.Number of Observation:196  
2.Summary Statistics:  
-Average of played_hours:5.85  
-Average of Age:21.14  
3.Number of Variables:7  
4.Name, Type of Variables, and What the Variables Mean:  
-experience(character):Gaming experience levels(e.g., Pro/Veteran/Amateur/Regular/Beginner)  
-subscribe(logical):The players subscribed or not  
-hashedEmail(character):Hashed version of player email used as an anonymous ID  
-played_hours(double):The time players played the game  
-name(character):The real name of the players  
-gender(character):Gender of the Players  
-Age(double):The age of the players  

-Issue  
Age contains missing value(NA)  
players.csv is collected per-players, whereas sessions.csv is collected per-sessions, so they have differences between the number of obserbations.  
-This data was collected by a research group in Computer Science at UBC, led by Frank Wood, is collecting data about how people play video games. They have set up a MineCraft serverLinks to an external site., and players' actions are recorded as they navigate through the world.

(2)Questions

This assignment tackle Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types and can players’ experience level, total played hours, gender, age predict whether they subscribe to the game-related newsletter? The players.csv dataset contains demographic imformation. Discribing the subscription rate by experience and gender and the proportion of subscription by the experience

(3) Exploratory Data Analysis and Visualization

In [ ]:
sessions_data <- read_csv("data/sessions.csv")
players_data <- read_csv("data/players.csv")

sessions_data
players_data

sessions|> summarise(average_original_start_time=mean(original_start_time,na.rm=TRUE), average_original_end_time=mean(original_end_time,na.rm=TRUE))|>round(2)
players |> summarise(average_played_hours=mean(played_hours, na.rm=TRUE), average_Age=mean(Age,na.rm=TRUE))|>round(2)

| Variable | Average |
|:-------|:------|
|Average of Original Start Time|1.719201e+12|
|Average of Original End Time|1.719196e+12|
|Average Played Hours|5.85|
|Average Age|21.14|

In [ ]:
library(dplyr)
library(ggplot2)
library(scales)
library(forcats)

colour_colection <- c("#0072B2", "#D55E00", "#009E73")

players_data <- players_data |>
mutate(experience = factor(experience, levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran")))

gender_mutate <- players_data |>
  mutate(gender_simple = recode(
    gender,
    "Male" = "Male",
    "Female" = "Female",
    "Agender" = "Others",
    "Non-binary" = "Others",
    "Prefer not to say" = "Others",
    "Two-Spirited" = "Others",
    "Other" = "Others"
  ))

subscription_rate_bar <- gender_mutate |> 
  group_by(experience, gender_simple) |> 
  summarise(subscribe_rate = mean(subscribe), .groups = "drop") |> 
  ggplot(aes(
    x = fct_reorder(experience, subscribe_rate, .desc = TRUE), 
    y = subscribe_rate, 
    fill = gender_simple
  )) +
  geom_col(position = "fill") +
  scale_fill_manual(values = colour_colection) +
  scale_y_continuous(labels = percent_format()) +
  labs(
    title = "Subscription Rate by Experience and Gender",
    x = "Experience Category",
    y = "Subscription Rate",
    fill="Gender"
  ) +
  theme_minimal() +
  theme(text = element_text(size = 12))

options(repr.plot.width = 12, repr.plot.height = 6)

subscription_rate_bar


subscribe_number_bar <- players_data|>
    group_by(experience, subscribe)|>
    summarise(count=n())|>
    ggplot(aes(x=experience, y=count, fill=subscribe))+
    geom_col(position="fill")+
    scale_y_continuous(labels = percent_format())+
    labs(title= "The Proportion of Subscription by Experience",
        x="Experience Category",
        y="The proportion of Subscribers")
subscribe_number_bar

This first bar graph illustrates the percentage of subscription rate of three different genders: male, female, and others within each experience category. Female players have the highest subscription rate, 40%, 39%, and 37.5% in regular, amateur, and veteran categories, respectively. It suggests that these three categories of females tend to subscribe to newsletters compared to other categories of females. 51% of the pro category’s subscribers are male players, and 49% are others. This suggests that the professional male players highly subscribe to it.  While approximately 45% of players who subscribe to the newsletter are others. Just beginner players have the highest rate, which other players subscribe to. In the veteran category, three types of gender equal proportion, 33%, subscribe to the newsletter.

The second bar graph illustrates the percentage of subscribers and non-subscribers by experience. Approximately 76% of players in the regular category subscribed to the newsletter, and the proportion is the highest compared to other categories.

Therefore, the female players in the regular category are the most likely to subscribe to it, because more than 75% of regular players subscribed, and 40% of subscribers in the regular category are female.


(4) Methods and Plan

I will use logistic regression to predict newsletter subscription because the response variable is binary and the method provides interpretable probability-based outputs. The model assumes independent observations, linearity of numeric predictors in the log-odds, and limited multicollinearity. Its main limitations include difficulty capturing non-linear relationships and sensitivity to class imbalance. I will split the dataset into 70% training and 30% testing, preprocess categorical and numeric variables, and apply 5-fold cross-validation on the training set.